In [63]:
import os
import numpy as np
import pandas as pd
from sklearn import neighbors, metrics 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split   

LOOKING INTO THE DATASETS:

In [64]:
#Importing the dataset
data_GTAV = pd.read_csv('FPS_grandTheftAuto5.csv') 
data_Fortnite = pd.read_csv('FPS_fortnite.csv') 
data_FarCry5 = pd.read_csv('FPS_farCry5.csv') 
data_CS = pd.read_csv('FPS_counterStrikeGlobalOffensive.csv') 
data_Battlefield4 = pd.read_csv('FPS_battlefield4.csv') 
data_PUBG = pd.read_csv('FPS_playerUnknownsBattlegrounds.csv') 
data_CODWW2 = pd.read_csv('FPS_callOfDutyWW2.csv') 

#printing the data just to check:
print(data_GTAV.head()) 
 

   CpuNumberOfCores  CpuNumberOfThreads  CpuBaseClock  CpuFrequency  CpuTDP  \
0                 8                  16           100          3600      95   
1                 8                   8           100          3600      95   
2                 6                  12           100          3700      95   
3                 4                   8           100          4200      91   
4                 6                   6           100          2900      65   

   CpuTurboClock  GpuBandwidth  GpuBaseClock  GpuBoostClock  GpuMemorySize  \
0           5000        616000          1350           1545          11000   
1           4900        616000          1350           1545          11000   
2           4700        616000          1350           1545          11000   
3           4500        616000          1350           1545          11000   
4           4100        616000          1350           1545          11000   

     FPS  
0  135.2  
1  134.0  
2  123.8  
3  120.2  
4

In [65]:
#Seperating the data in features and label
X_GTAV = data_GTAV.drop(columns = ['FPS']).values  
y_GTAV = data_GTAV['FPS'].values 

#Printing into arrays:
print("Features(X):\n", X_GTAV)
print("\nLabel(y):\n", y_GTAV)

Features(X):
 [[    8    16   100 ...  1350  1545 11000]
 [    8     8   100 ...  1350  1545 11000]
 [    6    12   100 ...  1350  1545 11000]
 ...
 [    6    12   100 ...  1168  1244  4000]
 [    6    12   100 ...  1168  1244  4000]
 [    8    16   100 ...  1168  1244  4000]]

Label(y):
 [135.2 134.  123.8 ...  40.3  39.8  38.6]


DOING THE SAME STUFF FOR ALL THE OTHER DATASETS

In [70]:
X_Fortnite = data_Fortnite.drop(columns = ['FPS']).values  
y_Fortnite = data_Fortnite['FPS'].values 

X_FarCry5 = data_FarCry5.drop(columns = ['FPS']).values  
y_FarCry5 = data_FarCry5['FPS'].values 

X_CS = data_CS.drop(columns = ['FPS']).values  
y_CS = data_CS['FPS'].values 

X_Battlefield4 = data_Battlefield4.drop(columns = ['FPS']).values  
y_Battlefield4 = data_Battlefield4['FPS'].values 

X_PUBG = data_PUBG.drop(columns = ['FPS']).values  
y_PUBG = data_PUBG['FPS'].values 

X_CODWW2 = data_CODWW2.drop(columns = ['FPS']).values  
y_CODWW2 = data_CODWW2['FPS'].values 


#Just checking on some random data:
print(X_FarCry5)

[[    8    16   100 ...  1350  1545 11000]
 [    8     8   100 ...  1350  1545 11000]
 [    6    12   100 ...  1350  1545 11000]
 ...
 [    6    12   100 ...  1168  1244  4000]
 [    6    12   100 ...  1168  1244  4000]
 [    8    16   100 ...  1168  1244  4000]]


NORMALISING DATA FOR MODEL'S CONVINIENCE:

We are having 10 features and all of them are of variating ranges so to just get them at a value of 0 to 1:
Taking the newest gen as a reference: i9-14900k paired with RTX 4090 (we didn't consider ryzen 9 7950X3D because it doesn't works on the p and e cores architecture)
No. of cores = 24
No. of threads = 32
CPUbaseClock = Assuming 100(won't affect the data)
CPUfrequency = 5600
CpuTDP = 125
Cpu_TURBO_CLOCK = 6000

GpuBandwidth = 1008*1000 = 1008000
GpuBaseClock = 2235 
GpuBoostClock = 2520
GpuMemorySize = 240000

In [72]:
#Normalising the data:

for i in range(0, 1026):
    X_GTAV[i][0] = X_GTAV[i][0]/24

for i in range(0, 1026):
    X_GTAV[i][1] = X_GTAV[i][1]/32

for i in range(0, 1026):
    X_GTAV[i][2] = X_GTAV[i][2]/100

for i in range(0, 1026):
    X_GTAV[i][3] = X_GTAV[i][3]/5600

for i in range(0, 1026):
    X_GTAV[i][4] = X_GTAV[i][4]/125

for i in range(0, 1026):
    X_GTAV[i][5] = X_GTAV[i][5]/6000

for i in range(0, 1026):
    X_GTAV[i][6] = X_GTAV[i][6]/1008000

for i in range(0, 1026):
    X_GTAV[i][7] = X_GTAV[i][7]/2235

for i in range(0, 1026):
    X_GTAV[i][8] = X_GTAV[i][8]/2520

for i in range(0, 1026):
    X_GTAV[i][9] = X_GTAV[i][9]/240000

#Normalising the label too:
#For GTAV Avg fps of data: 82.1423 but since it's locked at 185 fps for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_GTAV = y_GTAV/185


# Now checking the data again:
print(X_GTAV[23])
print(y_GTAV)


[0 0 0 0 0 0 0 0 0 0]
[0. 0. 0. ... 0. 0. 0.]


SPLITING THE DATA IN TRANING/TESTING:

In [57]:
X_GTAV_train, X_GTAV_test, y_GTAV_train, y_GTAV_test = train_test_split(X_GTAV, y_GTAV, test_size = 0.2)

#Doing same with the other data:
X_Fortnite_train, X_Fortnite_test, y_Fortnite_train, y_Fortnite_test = train_test_split(X_Fortnite, y_Fortnite, test_size = 0.2)
X_FarCry5_train, X_FarCry5_test, y_FarCry5_train, y_FarCry5_test = train_test_split(X_FarCry5, y_FarCry5, test_size = 0.2)
X_CS_train, X_CS_test, y_CS_train, y_CS_test = train_test_split(X_CS, y_CS, test_size = 0.2)
X_Battlefield4_train, X_Battlefield4_test, y_Battlefield4_train, y_Battlefield4_test = train_test_split(X_Battlefield4, y_Battlefield4, test_size = 0.2)
X_PUBG_train, X_PUBG_test, y_PUBG_train, y_PUBG_test = train_test_split(X_PUBG, y_PUBG, test_size = 0.2)
X_CODWW2_train, X_CODWW2_test, y_CODWW2_train, y_CODWW2_test = train_test_split(X_CODWW2, y_CODWW2, test_size = 0.2)


In [62]:
#Checking with the dimensions now:
#For Ex. data of GTAV
X_GTAV = np.array(X_GTAV)
print(X_GTAV)
print(y_GTAV)
print("\n", X_GTAV_train)
print(X_GTAV.shape)
print(X_GTAV_train.shape)
print(X_GTAV_train[0])


[[    8    16   100 ...  1350  1545 11000]
 [    8     8   100 ...  1350  1545 11000]
 [    6    12   100 ...  1350  1545 11000]
 ...
 [    6    12   100 ...  1168  1244  4000]
 [    6    12   100 ...  1168  1244  4000]
 [    8    16   100 ...  1168  1244  4000]]
[135.2 134.  123.8 ...  40.3  39.8  38.6]

 [[   4    4  100 ... 1506 1683 8000]
 [   6   12  100 ... 1465 1725 8000]
 [   4    4  100 ... 1469 1545 8000]
 ...
 [   6   12  100 ... 1291 1392 4000]
 [   6   12  100 ... 1469 1545 8000]
 [   6    6  100 ... 1291 1392 4000]]
(1026, 10)
(820, 10)
[     4      4    100   3800     91   4200 256300   1506   1683   8000]
